In [3]:
import sys
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pandas_profiling import ProfileReport
from ipywidgets import widgets

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
